# Examining 2012 Oregon Election Results

Using data from OpenElections' [Oregon repository](https://github.com/openelections/openelections-data-or), we'll look at some aggregates and patterns for specific offices and counties. First we'll save the [raw version of the 2012 county-level results file](https://raw.githubusercontent.com/openelections/openelections-data-or/master/2012/20121106__or__general.csv) locally and load it into Agate. 

In [1]:
import agate

In [2]:
results = agate.Table.from_csv('20121106__or__general.csv')

In [3]:
print results

|---------------+---------------|
|  column_names | column_types  |
|---------------+---------------|
|  county       | Text          |
|  office       | Text          |
|  district     | Number        |
|  party        | Text          |
|  candidate    | Text          |
|  votes        | Number        |
|---------------+---------------|



Next we'll take a look at results by office and party. We'll group by office first, then by party, and add up the votes for each combination of office and party statewide.

In [15]:
offices = results.where(lambda r: r['candidate'] != 'Under Votes' and r['candidate'] != 'Over Votes').group_by('office')

In [16]:
office_and_party = offices.group_by('party')

In [17]:
op_totals = office_and_party.aggregate([('vote_total', agate.Sum('votes'))])

In [18]:
op_totals = op_totals.order_by('vote_total', reverse=True)

In [19]:
op_totals.print_table()

|---------------------+-------+-------------|
|  office             | party | vote_total  |
|---------------------+-------+-------------|
|  President          | D     |    970,488  |
|  State Treasurer    | D     |    955,213  |
|  U.S. House         | D     |    929,511  |
|  Attorney General   | D     |    898,770  |
|  Secretary of State | D     |    863,656  |
|  State House        | D     |    768,193  |
|  President          | R     |    754,175  |
|  State House        | R     |    741,110  |
|  Secretary of State | R     |    727,607  |
|  U.S. House         | R     |    669,565  |
|  Attorney General   | R     |    620,305  |
|  State Treasurer    | R     |    609,989  |
|  State Senate       | D     |    421,250  |
|  State Senate       | R     |    385,624  |
|  Secretary of State | PG    |     44,235  |
|  Attorney General   | C     |     44,209  |
|  State Treasurer    | P     |     38,762  |
|  State Treasurer    | L     |     30,002  |
|  Attorney General   | P     |   

No surprises here: Democrats got more popular votes for every type of office, with Barack Obama leading the way in the presidential race. But that's statewide; are there counties where Obama ran behind other Democrats? We'll group by office, party and county and calculate a vote total.

In [20]:
office_party_county = office_and_party.group_by('county')

In [21]:
office_party_county_totals = office_party_county.aggregate([('vote_total', agate.Sum('votes'))])

In [22]:
office_party_county_totals = office_party_county_totals.order_by('vote_total', reverse=True)

In [23]:
office_party_county_totals.print_table()

|---------------------+-------+------------+-------------|
|  office             | party | county     | vote_total  |
|---------------------+-------+------------+-------------|
|  President          | D     | Multnomah  |    274,887  |
|  U.S. House         | D     | Multnomah  |    268,714  |
|  State Treasurer    | D     | Multnomah  |    261,764  |
|  Attorney General   | D     | Multnomah  |    253,398  |
|  State House        | D     | Multnomah  |    240,572  |
|  Secretary of State | D     | Multnomah  |    230,643  |
|  State Senate       | D     | Multnomah  |    204,330  |
|  President          | D     | Washington |    135,291  |
|  U.S. House         | D     | Washington |    132,119  |
|  State Treasurer    | D     | Washington |    130,584  |
|  Attorney General   | D     | Washington |    124,761  |
|  State House        | D     | Washington |    120,849  |
|  Secretary of State | D     | Washington |    115,639  |
|  U.S. House         | D     | Lane       |    113,547 

Indeed there are. In both Lane and Clackamas counties (at least), total votes for Democrats seeking other offices exceeded the presidential vote. Let's see if we can't find all of them. What we want to do is to find counties where the Democratic vote for President ran behind other offices' Democratic total. First, let's filter on Democratic candidates:

In [24]:
democratic_office_totals = office_party_county_totals.where(lambda r: r['party'] == 'D')

Then we'll create a collection of presidential results:

In [25]:
president_by_county = democratic_office_totals.where(lambda r: r['office'] == 'President')

In [26]:
print president_by_county.column_names

(u'office', u'party', u'county', 'vote_total')


And then a collection of non-presidential offices and their county vote totals:

In [27]:
others_by_county = democratic_office_totals.where(lambda r: r['office'] != 'President')

Finally, we can find counties where non-presidential offices had higher vote totals than Obama:

In [28]:
more_than_prez = democratic_office_totals.where(lambda r: r['vote_total'] > [x['vote_total'] for x in president_by_county.rows 
                                                                           if r['county'] == x['county']][0])

In [29]:
more_than_prez.print_table(max_rows=50)

|---------------------+-------+-----------+-------------|
|  office             | party | county    | vote_total  |
|---------------------+-------+-----------+-------------|
|  U.S. House         | D     | Lane      |    113,547  |
|  U.S. House         | D     | Clackamas |     98,879  |
|  State Treasurer    | D     | Clackamas |     98,039  |
|  State Treasurer    | D     | Marion    |     58,948  |
|  U.S. House         | D     | Marion    |     58,417  |
|  U.S. House         | D     | Benton    |     28,752  |
|  U.S. House         | D     | Linn      |     23,747  |
|  U.S. House         | D     | Douglas   |     22,954  |
|  State Treasurer    | D     | Linn      |     21,347  |
|  Attorney General   | D     | Linn      |     20,556  |
|  Secretary of State | D     | Linn      |     20,541  |
|  State Treasurer    | D     | Yamhill   |     19,989  |
|  State Treasurer    | D     | Douglas   |     18,309  |
|  Secretary of State | D     | Douglas   |     18,022  |
|  Attorney Ge

Many of these counties are have small populations and are not prime political ground for a Democratic president. But Ted Wheeler, the Democratic state treasurer, out-performed an incumbent president of the same party in at least 18 counties in the state, including Clackamas and Marion. Other offices did the same, particularly in more rural areas.

The next question: is that typical for Oregon? For other states?